# Milestone 1: Fitness Data Preprocessing and Normalization

# Objective:
The goal of this notebook is to ingest raw fitness tracker data (CSV files), validate schemas, and normalize the data into a clean, consolidated format for analysis.

# Key Tasks:

Ingest: Read CSV files for Sleep, Steps, and Heart Rate.

Validate: Ensure column names and data types are consistent.

Normalize: Convert all timestamps to UTC.

Align: Resample disparate data frequencies to a uniform 1-minute interval.

Clean: Handle missing values and generate a final output file.

#  Imports and Setup
First, we import the necessary libraries and load the raw dataset. We will also perform an initial inspection to understand the data structure.

In [39]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Display settings to ensure we can see all columns during inspection
pd.set_option('display.max_columns', None)

print("Libraries loaded successfully.")

Libraries loaded successfully.


# Data Ingestion & Schema Validation


In [40]:
print("--- Step 1: Ingesting Data ---")

try:
    # Load the datasets with low_memory=False to handle mixed types efficiently
    sleep_df = pd.read_csv("/content/minuteSleep_merged.csv", low_memory=False)
    steps_df = pd.read_csv("/content/minuteStepsNarrow_merged.csv", low_memory=False)
    hr_df = pd.read_csv("/content/heartrate_seconds_merged.csv", on_bad_lines="skip", low_memory=False)

    # 1. Standardize Column Names
    # We map dataset-specific time columns to a generic 'Time' column
    sleep_df = sleep_df.rename(columns={"date": "Time", "value": "Sleep"})
    steps_df = steps_df.rename(columns={"ActivityMinute": "Time"})
    hr_df = hr_df.rename(columns={"Value": "HeartRate"})

    # 2. Enforce Data Types
    # IDs must be strings to prevent numerical errors during merging
    for df in [sleep_df, steps_df, hr_df]:
        df["Id"] = df["Id"].astype(str)

    print(" Datasets loaded and schema standardized.")
    print(f"  - Sleep Data Shape: {sleep_df.shape}")
    print(f"  - Steps Data Shape: {steps_df.shape}")
    print(f"  - Heart Rate Data Shape: {hr_df.shape}")

except FileNotFoundError:
    print(" Error: One or more files are missing.")
    print("Please ensure 'minuteSleep_merged.csv', 'minuteStepsNarrow_merged.csv', and 'heartrate_seconds_merged.csv' are in /content/.")

--- Step 1: Ingesting Data ---
 Datasets loaded and schema standardized.
  - Sleep Data Shape: (188521, 4)
  - Steps Data Shape: (1445040, 3)
  - Heart Rate Data Shape: (2483658, 3)


# Pre-Processing Aggregation


In [41]:
print("--- Step 2: Aggregating to 1-Minute Intervals ---")

# We specify the format explicitly to fix the warning and speed up processing
# Format: Month/Day/Year Hour:Minute:Second AM/PM
date_fmt = "%m/%d/%Y %I:%M:%S %p"

# Convert Time columns to datetime objects using the format
sleep_df["Time"] = pd.to_datetime(sleep_df["Time"], format=date_fmt, errors="coerce")
steps_df["Time"] = pd.to_datetime(steps_df["Time"], format=date_fmt, errors="coerce")

# For Heart Rate, we parse then floor to the nearest minute
hr_df["Time"] = pd.to_datetime(hr_df["Time"], format=date_fmt, errors="coerce").dt.floor("min")

# Drop rows where Time conversion might have failed
sleep_df = sleep_df.dropna(subset=["Time"])
steps_df = steps_df.dropna(subset=["Time"])
hr_df = hr_df.dropna(subset=["Time"])

# Group by ID and Time to handle duplicates (Aggregation)
steps_agg = steps_df.groupby(["Id", "Time"], as_index=False)["Steps"].sum()
sleep_agg = sleep_df.groupby(["Id", "Time"], as_index=False)["Sleep"].max()
hr_agg = hr_df.groupby(["Id", "Time"], as_index=False)["HeartRate"].mean()

print("✓ Data aggregated successfully.")
print(f"  - Aggregated HR Shape: {hr_agg.shape}")

--- Step 2: Aggregating to 1-Minute Intervals ---
✓ Data aggregated successfully.
  - Aggregated HR Shape: (333420, 3)


# Merging & Normalization


In [42]:
print("--- Step 3: Merging & UTC Normalization ---")

# Merge Steps and Sleep first
merged_temp = steps_agg.merge(sleep_agg, on=["Id", "Time"], how="inner")

# Merge the result with Heart Rate
merged_df = merged_temp.merge(hr_agg, on=["Id", "Time"], how="inner")

# Normalize to UTC
merged_df["Time"] = pd.to_datetime(merged_df["Time"], utc=True)

print(f"✓ Data merged. Combined Shape: {merged_df.shape}")
print(f"  - Date Range: {merged_df['Time'].min()} to {merged_df['Time'].max()}")

--- Step 3: Merging & UTC Normalization ---
✓ Data merged. Combined Shape: (1095, 5)
  - Date Range: 2016-04-12 00:00:00+00:00 to 2016-04-12 09:59:00+00:00


# Data Cleaning (Missing Values & Alignment)


In [43]:
print("--- Step 4: Cleaning & Resampling ---")

# 1. Fill NaN values
merged_df["Steps"] = pd.to_numeric(merged_df["Steps"], errors="coerce").fillna(0)
merged_df["Sleep"] = pd.to_numeric(merged_df["Sleep"], errors="coerce").fillna(0)
median_hr = merged_df["HeartRate"].median()
merged_df["HeartRate"] = pd.to_numeric(merged_df["HeartRate"], errors="coerce").fillna(median_hr)

# 2. Resample to ensure continuous 1-minute intervals
# Set Index for resampling
merged_df = merged_df.set_index("Time")

# Resample by ID
final_df = merged_df.groupby("Id").resample("1min").agg({
    "Steps": "sum",
    "Sleep": "max",
    "HeartRate": "mean"
})

# Resampling creates NaNs for the new time slots (gaps), fill them again
final_df["Steps"] = final_df["Steps"].fillna(0)
final_df["Sleep"] = final_df["Sleep"].fillna(0)
final_df["HeartRate"] = final_df["HeartRate"].fillna(median_hr)

# Reset index to restore columns
final_df = final_df.reset_index()

print("✓ Data cleaned and realigned to 1-minute intervals.")

--- Step 4: Cleaning & Resampling ---
✓ Data cleaned and realigned to 1-minute intervals.


# Final Output

In [44]:
print("--- Step 5: Final Output ---")

# Sort for better readability
final_df = final_df.sort_values(["Id", "Time"])

# Save to CSV
output_path = "/content/fitbit_clean_final.csv"
final_df.to_csv(output_path, index=False)

print(f"✓ Success! File saved to: {output_path}")
print("\nFirst 5 rows of clean data:")
print(final_df.head())

--- Step 5: Final Output ---
✓ Success! File saved to: /content/fitbit_clean_final.csv

First 5 rows of clean data:
           Id                      Time  Steps  Sleep  HeartRate
0  4020332650 2016-04-12 00:00:00+00:00      0    1.0  63.500000
1  4020332650 2016-04-12 00:01:00+00:00      0    1.0  65.913043
2  4020332650 2016-04-12 00:02:00+00:00      0    1.0  67.000000
3  4020332650 2016-04-12 00:03:00+00:00      0    1.0  67.000000
4  4020332650 2016-04-12 00:04:00+00:00      0    1.0  67.000000
